# 🌾 Notebook 3: Modeling & Forecasting
# ==========================================

In [2]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

### Load processed data

In [3]:
df = pd.read_csv("/content/processed_data.csv")

### Filter for a specific country & crop

In [4]:
country = "India"
crop = "Wheat"
dff = df[(df["Area"] == country) & (df["Crop"] == crop)]


### Prepare for Prophet

In [ ]:
prophet_df = dff[["Year", "Yield"]].rename(columns={"Year": "ds", "Yield": "y"})
prophet_df["ds"] = pd.to_datetime(prophet_df["ds"], format="%Y")
prophet_df = prophet_df.groupby("ds").mean().reset_index()

### Train model

In [6]:
model = Prophet()
model.fit(prophet_df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lmwwgqx/33vefics.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1lmwwgqx/xiyaua4h.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=94187', 'data', 'file=/tmp/tmp1lmwwgqx/33vefics.json', 'init=/tmp/tmp1lmwwgqx/xiyaua4h.json', 'output', 'file=/tmp/tmp1lmwwgqx/prophet_model4lbpva4i/prophet_model-20250914001127.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:11:27 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:11:27 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


### Forecast

In [7]:
future = model.make_future_dataframe(periods=10, freq="Y")
forecast = model.predict(future)

/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(


### Plot forecast

In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=prophet_df["ds"], y=prophet_df["y"], mode="lines+markers", name="Actual"))
fig.add_trace(go.Scatter(x=forecast["ds"], y=forecast["yhat"], mode="lines", name="Forecast"))
fig.update_layout(title=f"📈 Yield Forecast for {crop} in {country}")
fig.show()

### Evaluate

In [12]:
y_true = prophet_df["y"]
y_pred = forecast["yhat"][:len(y_true)]
print("MAE:", mean_absolute_error(y_true, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))

MAE: 40.97711817985502
RMSE: 86.52624758382187
